In [ ]:
import requests  # Import the requests library
import gspread  # Google sheet

import pandas as pd
from google.colab import auth
from oauth2client.client import GoogleCredentials

##Total Star count of repositories

In [ ]:


def getRepoStarCount(url):
  user = url.split('https://github.com/')[1].split('/')[0]
  repoName = url.split('https://github.com/')[1].split('/')[1]
  api_url = 'https://api.github.com/repos/' + user + '/' + repoName
  
  response = requests.get(api_url)  # Make a GET request to the URL
  print(response.json())
  
  # Print status code (and associated text)
  # print(f"Request returned {response.status_code} : '{response.reason}'")

  payload = response.json()  # Parse `response.text` into JSON
  starCount = payload['stargazers_count']
  return starCount

print(getRepoStarCount('https://github.com/e32wong/CloCom/blob/master/MatchGroup.java'))


{'id': 26790529, 'node_id': 'MDEwOlJlcG9zaXRvcnkyNjc5MDUyOQ==', 'name': 'CloCom', 'full_name': 'e32wong/CloCom', 'private': False, 'owner': {'login': 'e32wong', 'id': 9814980, 'node_id': 'MDQ6VXNlcjk4MTQ5ODA=', 'avatar_url': 'https://avatars.githubusercontent.com/u/9814980?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/e32wong', 'html_url': 'https://github.com/e32wong', 'followers_url': 'https://api.github.com/users/e32wong/followers', 'following_url': 'https://api.github.com/users/e32wong/following{/other_user}', 'gists_url': 'https://api.github.com/users/e32wong/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/e32wong/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/e32wong/subscriptions', 'organizations_url': 'https://api.github.com/users/e32wong/orgs', 'repos_url': 'https://api.github.com/users/e32wong/repos', 'events_url': 'https://api.github.com/users/e32wong/events{/privacy}', 'received_events_url': 'https://api.github.com/

In [ ]:
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())


In [ ]:
worksheet1 = gc.open('java_commented_file_urls').sheet1

# get_all_values gives a list of rows.
rows = worksheet1.get_all_values()
no_of_rows = len(rows)

url_list = list()
for row in rows:
  url_list.append(row[0])


corresponding_star_count_list = []
for url in url_list:
  c = getRepoStarCount(url)
  corresponding_star_count_list.append(c)

final_B_cell = 'B'+ str(no_of_rows)
range_of_B = 'B1:' + final_B_cell
worksheet1.update_cells(range_of_B, corresponding_star_count_list )


In [ ]:
print(corresponding_star_count_list)

[130, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]


# Comment Extracting to a list

In [ ]:
# Load the url containing google sheet as a csv
sheet_id = '1VdY3FKrtxA46N49qf_T6WJwg8b-jaUXARU7rc3AbLQY'
sheet_name = 'Sheet1'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
url_dataset = pd.read_csv(url)
url_dataset.head()


,url,star_count
0,https://github.com/apache/commons-validator/bl...,131
1,https://github.com/e32wong/CloCom/blob/master/...,14
2,https://github.com/e32wong/CloCom/blob/master/...,14
3,https://github.com/e32wong/CloCom/blob/master/...,14
4,https://github.com/e32wong/CloCom/blob/master/...,14


In [ ]:
!pip install wget
!pip install comment_parser


In [ ]:
from datetime import datetime
import wget

from comment_parser import comment_parser
from comment_parser import parsers
# https://github.com/jeanralphaviles/comment_parser

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os.path
from os import path


if path.exists('/content/tempo_code_files') == False:
  os.mkdir('/content/tempo_code_files')

Mounted at /content/drive


In [ ]:
#Create github raw URL from a normal page URL
def get_raw_url(url):
  url = url.replace('github.com', 'raw.githubusercontent.com', 1)
  url = url.replace('/blob/', '/',1)
  return url;


# Download each file as a java file and extract full comment detail list
def get_full_comment_list_of_a_file(url):
  """
  Attributes
  -----------
  url: str
     raw-url of a java file and the url must end with '.java' extension.
  
  Returns
  --------
  A list containing comment units(comment,line_number,isMultiline)
  """
  #Creating file name
  now_time =datetime.now()
  millisec = now_time.timestamp() * 10000
  millisec = str(millisec).split('.')[0]
  partial_name = url.split('/')[-1].split('.')[0]
  file_name = partial_name + millisec + '.java'

  #Download the file and save in colab location
  wget.download(url, f"tempo_code_files/{file_name}")

  saved_file = f"/content/tempo_code_files/{file_name}"
  z = comment_parser.extract_comments(saved_file, mime='text/x-java-source')
  
  return z, saved_file



In [ ]:
repeated_saved_file = []
comment_text_list = []
comment_starting_line_number_list = []
comment_is_multiLine = []

# Create a new dataset with above details
new_dataset_1 = pd.DataFrame(columns=['saved_file','comment_text','starting_line_number','is_multiLine'])

for url in url_dataset['url']:
  raw_url = get_raw_url(url)
  
  comment_list_of_a_file, saved_file = get_full_comment_list_of_a_file(raw_url)
  
  

  for full_comment in comment_list_of_a_file:
    repeated_saved_file.append(raw_url)
    # print('\n-------------------------------------\n')
    # print(full_comment)
    # print(str(full_comment.line_number).split(',')[-2])
    # print(str(full_comment.line_number).split(',')[-1].split(')')[0])

    comment_text_list.append(full_comment)
    comment_starting_line_number_list.append(int(str(full_comment.line_number).split(',')[-2]))
    comment_is_multiLine.append(bool(str(full_comment.line_number).split(',')[-1].split(')')[0]))

  
  
  
new_dataset_1['saved_file'] = repeated_saved_file
new_dataset_1['comment_text'] = comment_text_list
new_dataset_1['starting_line_number'] = comment_starting_line_number_list
new_dataset_1['is_multiLine'] = comment_is_multiLine

print(len(comment_text_list))
  
# Save the dataset to a csv file
new_dataset_1.to_csv('url_with_extracted_comments.csv', index=False)


696


In [ ]:
#Save the file to the drive
!cp -av /content/url_with_extracted_comments.csv /content/drive/MyDrive/Shared_with_me/Research_Project_Team_Bro_Code/Dataset_preparation

'/content/url_with_extracted_comments.csv' -> '/content/drive/MyDrive/Shared_with_me/Research_Project_Team_Bro_Code/Dataset_preparation/url_with_extracted_comments.csv'
